上から順番に再生ボタンを押して行ってください。下2つは自分のbotの設定を入れてください。

In [ ]:
!pip install transformers accelerate bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16,device_map="auto")

In [5]:
def genText(text):
  with torch.no_grad():
    token_ids = tokenizer.encode(text,add_special_tokens=False,return_tensors="pt")
    output_ids = model.generate(token_ids.to(model.device),max_new_tokens=50,pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id,)
  return tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

In [30]:
import json
import requests
import time
import urllib

In [34]:
def chat(text):
  setting = "あなたはかわいらしい口調で話すだいふくという名前のアシスタントです。"#@param {type:"string"}
  return genText("<s>[INST] <<SYS>>\n"+ setting +"\n<</SYS>>\n\n" + text +" [/INST] ").replace("\n","")

In [ ]:
t = json.loads(requests.get("https://yukibbs-server.onrender.com/bbs/bot").text)
n = t[0]
seed = "\u3053\u3053\u306Bbot\u306E\u30B7\u30FC\u30C9\u3092\u5165\u529B"#@param {type:"string"}
name = "\u3060\u3044\u3075\u304F"#@param {type:"string"}
while True:
  t = json.loads(requests.get("https://yukibbs-server.onrender.com/bbs/bot").text)
  if n == t[0] or t[1][-1] == ":":
   time.sleep(1)
   continue
  n = t[0]
  requests.get(fr"https://yukibbs-server.onrender.com/bbs/result?name={urllib.parse.quote(name)}&message={n}+{urllib.parse.quote(chat(t[1]))}:&seed={urllib.parse.quote(seed)}&channel=main",cookies={"yuki":"True"})